In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Step 1: Fetch the webpage
url = 'https://data.timesrecordnews.com/weather-data/indiana/18/1920-06-01/table/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
# Step 2: Locate the table
# This selector may vary based on the webpage's HTML structure
table = soup.find('table')

In [4]:
# Step 3: Extract data
data = []
headers = [header.text for header in table.find_all('th')]
for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        data.append([cell.text.strip() for cell in cells])

In [6]:
# Step 4: Save to DataFrame and Export
df = pd.DataFrame(data, columns=headers)
df.to_csv('table_data.csv', index=False)  # Save as CSV

In [7]:
df = pd.read_csv('table_data.csv')
df['Month'] = df['Month'].str[8:]
# Split 'Month' column into 'month' and 'year' columns
df[['Month', 'Year']] = df['Month'].str.split(',', expand=True)

# Remove any whitespace and convert 'year' to integer
df['Month'] = df['Month'].str.strip()
df['Year'] = df['Year'].str.strip().astype(int)\

df = df[['Month', 'Year','Avg. Temp.','Min. Temp.','Max. Temp.','Precipitation','Heating Degree Days','Cooling Degree Days']]
df = df[df['Year']>=1940]

print(df)

         Month  Year  Avg. Temp.  Min. Temp.  Max. Temp.  Precipitation  \
0         July  2024        73.2        62.8        83.6           5.11   
1         June  2024        72.9        61.2        84.3           2.80   
2          May  2024        66.6        55.6        77.7           4.77   
3        April  2024        54.8        44.2        65.4           6.67   
4        March  2024        46.3        35.5        57.2           2.78   
...        ...   ...         ...         ...         ...            ...   
1010       May  1940        57.9        46.6        69.3           4.27   
1011     April  1940        47.6        36.5        58.8           5.40   
1012     March  1940        35.8        27.1        44.4           1.93   
1013  February  1940        29.5        22.9        36.2           2.38   
1014   January  1940        14.1         5.5        22.7           1.63   

     Heating Degree Days  Cooling Degree Days  
0                      6                249.0  
1  

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import time

In [21]:
# Initialize the Chrome WebDriver
service = Service("C:\Users\nasod\webdriver\chrome-win64")  # Specify the path to your ChromeDriver
driver = webdriver.Chrome(service=service)

# URL to the website containing the tables
url = "https://data.timesrecordnews.com/weather-data/adams-county-indiana/18001/1895-01-01/?syear=1895&eyear=2024#compare"  # Replace with the actual URL

# Open the webpage
driver.get(url)
time.sleep(2)  # Wait for page to load

# Locate the dropdown element
dropdown = driver.find_element(By.ID, "cyms")  # Replace with the actual dropdown element ID

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1188038709.py, line 2)

In [25]:
driver = webdriver.Chrome()  # Optional argument, if not specified will search path.
driver.get('https://data.timesrecordnews.com/weather-data/adams-county-indiana/18001/1895-01-01/?syear=1895&eyear=2024#compare');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name(By.ID, "cyms")
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_name'

In [31]:
# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver,10)

# URL to the website containing the tables
url = "https://data.timesrecordnews.com/weather-data/adams-county-indiana/18001/1940-01-01/?syear=1895&eyear=2024#compare"  # Replace with the actual URL

# Open the webpage  
driver.get(url)
time.sleep(2)  # Wait for page to load

# Locate the dropdown menu and create a Select object
dropdown = wait.until(EC.presence_of_element_located((By.ID, "cyms")))  # Update dropdown_id as needed
select = Select(dropdown)

# Store all dropdown options in a list (text only) to avoid re-fetching after page refresh
options_text = [option.text for option in select.options]

# Use Select to iterate through each option in the dropdown
all_data = pd.DataFrame()  

# Loop through each option in the dropdown by its text
for option_text in options_text:
    # Re-select the dropdown element on each loop iteration
    dropdown = wait.until(EC.presence_of_element_located((By.ID, "cyms")))
    select = Select(dropdown)
    select.select_by_visible_text(option_text)

    # Wait for the page to load or the URL to change if necessary
    time.sleep(2)  # Adjust sleep time based on page load speed
    
    # Refresh the URL to ensure it’s loaded (if each option uses a new URL)
    driver.get(driver.current_url)
    
    # Wait for the page to load or the URL to change if necessary
    time.sleep(2)  # Adjust sleep time based on page load speed
    
    # Refresh the URL to ensure it’s loaded (if each option uses a new URL)
    driver.get(driver.current_url)
    
    # Locate the table and extract rows
    table = wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # Extract table data
    table_data = []
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        if row_data:  # Skip empty rows
            table_data.append(row_data)

    # Create a DataFrame from the extracted data and append to the main DataFrame
    df = pd.DataFrame(table_data)
    df['Source'] = option_text  # Add column to identify the data source
    all_data = pd.concat([all_data, df], ignore_index=True)

# Close the WebDriver
driver.quit()

# Save all data to a CSV file
all_data.to_csv("all_tables_data.csv", index=False)
print("Data saved to all_tables_data.csv")

Data saved to all_tables_data.csv


In [32]:
from IPython.display import FileLink

In [37]:
df1 = all_data
df1

,0,1,2,3,4,5,6,Source
0,}\nJan. 1895\n}\nFeb. 1895\n}\nMarch 1895\n}\n...,19.80,11.30,28.40,2.50,,,"Jan., 1895"
1,1901-2000 Avg. (Jan. Only),24.50,16.78,32.23,2.31,,,"Jan., 1895"
2,Historic High (Jan. Only),37.80 (2006),31.30 (2006),44.30 (2006),9.80 (1950),"1,337 (2022)",0 (2024),"Jan., 1895"
3,Historic Low (Jan. Only),8.60 (1977),-0.20 (1977),17.30 (1977),0.36 (1961),934 (2023),0 (2021),"Jan., 1895"
4,1901-2000 Avg. (All Months),49.54,39.45,59.63,3.00,947,,"Jan., 1895"
...,...,...,...,...,...,...,...,...
10880,Historic High (July Only),79.30 (1934),67.20 (2011),92.70 (1934),11.18 (1896),0 (2024),277 (2022),"July, 2024"
10881,Historic Low (July Only),68.50 (2014),57.60 (1918),79.10 (2014),0.60 (1974),0 (2021),249 (2024),"July, 2024"
10882,1901-2000 Avg. (All Months),49.54,39.45,59.63,3.00,947,,"July, 2024"
10883,Historic High (All Months),"79.30 (July, 1934)","67.30 (Aug., 1995)","92.70 (July, 1934)","13.32 (June, 2015)","1,482 (Dec., 1989)","294 (Aug., 2021)","July, 2024"


In [39]:
df1.to_csv('df1.csv', index=False)